<a href="https://colab.research.google.com/github/AnshuMishra01/Ensemble-based-hotel-recommender-system/blob/main/class3_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
%pip install -q -U watermark

In [ ]:
%pip install -qq transformers

In [ ]:
%pip install torch

In [ ]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch, transformers

In [ ]:
%pip freeze

absl-py==1.4.0
aiohttp==3.9.3
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.6.0
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array-record==0.5.0
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.14.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==0.22.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
branca==0.7.1
build==1.1.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
certifi==2024.2.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.85
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.1
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.4
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.0
cryptography==42.0.5
cufflinks==0.17.3
cupy-cuda12x==12.2.0


In [ ]:
%pip install tensorflow-gpu==2.8.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.1/498.1 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 50.1 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2

In [ ]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

%matplotlib inline
%config InlineBackend.figure_format='retina'

def swish(x):
    return x * tf.nn.sigmoid(x)

sns.set(style='whitegrid',palette='muted',font_scale=1.2)

PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(PALETTE))
rcParams['figure.figsize'] = 12,8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

device


device(type='cuda', index=0)

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
df=pd.read_csv("/content/review.csv",encoding='ISO-8859–1',header=0)
df.head()

,ï»¿text,Unnamed: 1,Sentimental_1,Score,Neutral,Sentimental
0,"This hotel was nice and quiet. Did not know, t...",NaN,positive,0.69,False,positive
1,We stayed in the king suite with the separatio...,NaN,negative,0.01,False,negative
2,"Parking was horrible, somebody ran into my ren...",NaN,negative,0.07,False,negative
3,Not cheap but excellent location. Price is som...,NaN,positive,0.93,False,positive
4,If you get the room that they advertised on th...,NaN,negative,0.39,False,negative


In [ ]:
df.rename(columns={df.columns[0]: 'text', df.columns[5]: 'sentimental'}, inplace=True)

In [ ]:
df1 = df.drop(['Unnamed: 1','Sentimental_1','Score','Neutral'], axis=1)
df1.head()

,text,sentimental
0,"This hotel was nice and quiet. Did not know, t...",positive
1,We stayed in the king suite with the separatio...,negative
2,"Parking was horrible, somebody ran into my ren...",negative
3,Not cheap but excellent location. Price is som...,positive
4,If you get the room that they advertised on th...,negative


In [ ]:
df1.loc[ df1['sentimental'] =='positive', 'sentimental'] = 2
df1.loc[ df1['sentimental'] =='negative', 'sentimental'] = 0
df1.loc[ df1['sentimental'] =='neutral', 'sentimental'] = 1
df1.head()

,text,sentimental
0,"This hotel was nice and quiet. Did not know, t...",2
1,We stayed in the king suite with the separatio...,0
2,"Parking was horrible, somebody ran into my ren...",0
3,Not cheap but excellent location. Price is som...,2
4,If you get the room that they advertised on th...,0


In [ ]:
PRE_TRAINED_MODEL_NAME='bert-base-uncased'

In [ ]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
MAX_LEN = 160

In [ ]:
class hotelDataset(Dataset):

  def __init__(self, reviews, targets, tokenizer, max_len):
    self.reviews = reviews
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.reviews)

  def __getitem__(self, item):
    review = str(self.reviews[item])
    target = self.targets[item]

    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    return {
      'text': review,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [ ]:
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
df_train, df_val, df_test = np.split(df1.sample(frac=1), [int(.7*len(df1)), int(.9*len(df1))])

In [ ]:
def create_data_loader(df1, tokenizer, max_len, batch_size):
  ds = hotelDataset(
    reviews=df1.text.to_numpy(),
    targets=df1.sentimental.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4
  )

In [ ]:
BATCH_SIZE = 16

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:

class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  def forward(self, input_ids, attention_mask):
      outputs = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask
      )
      pooled_output = outputs.pooler_output  # Accessing the pooled output directly from 'outputs'
      output = self.drop(pooled_output)
      logits = self.out(output)
      return logits

In [ ]:
sample_txt = 'When was I last outside? I am stuck at home for 2 weeks.'
encoding = tokenizer.encode_plus(
  sample_txt,
  max_length=32,
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',  # Return PyTorch tensors
)

encoding.keys()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


dict_keys(['input_ids', 'attention_mask'])

In [ ]:
last_hidden_state, pooled_output = bert_model(
  input_ids=encoding['input_ids'],
  attention_mask=encoding['attention_mask']
)

In [ ]:
print(input_ids)
print(attention_mask)

tensor([[  101,  2057, 17414,  ...,     0,     0,     0],
        [  101,  2057,  2020,  ...,     0,     0,     0],
        [  101,  1996,  3309,  ...,     0,     0,     0],
        ...,
        [  101,  2108,  1037,  ...,  2063,  8214,   102],
        [  101,  2057,  2020,  ...,     0,     0,     0],
        [  101,  4370,  2182,  ...,  2115,  3791,   102]], device='cuda:0')
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0')


In [ ]:
class_names = ['negative', 'neutral', 'positive']

In [ ]:
model = SentimentClassifier(len(class_names))
model = model.to(device)

In [ ]:
model(input_ids, attention_mask)

tensor([[ 0.5365, -0.3212, -0.2324],
        [ 0.6234,  0.1272, -0.0185],
        [ 0.4370, -0.5954,  0.0810],
        [ 0.7547,  0.1062,  0.0819],
        [ 0.5939,  0.1023,  0.1251],
        [ 0.2997, -0.3032, -0.1698],
        [ 0.3243, -0.3796,  0.1343],
        [ 0.5386, -0.4015, -0.6934],
        [-0.0143, -0.0745, -0.3553],
        [ 0.5739, -0.4862, -0.0112],
        [-0.1159, -0.1254,  0.0429],
        [ 0.7009, -0.1722,  0.0650],
        [-0.0596, -0.3991,  0.2933],
        [ 0.4200, -0.5347, -0.4404],
        [ 0.0089, -0.3637, -0.3058],
        [ 0.7175, -0.1247,  0.0073]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [ ]:
data = next(iter(train_data_loader))
data.keys()

dict_keys(['text', 'input_ids', 'attention_mask', 'targets'])

In [ ]:
input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)

print(input_ids.shape) # batch size x seq length
print(attention_mask.shape) # batch size x seq length

torch.Size([16, 160])
torch.Size([16, 160])


In [ ]:
model(input_ids, attention_mask)

tensor([[ 0.4773, -0.4443, -0.0273],
        [ 0.3259, -0.4563, -0.3994],
        [ 0.4038, -0.3464, -0.6828],
        [ 0.5894, -0.0714, -0.3222],
        [ 0.5140, -0.1289, -0.5824],
        [ 0.5049, -0.2581, -0.1498],
        [ 0.2234,  0.1403,  0.1163],
        [ 0.6230, -0.1814,  0.1931],
        [ 0.4036,  0.0928,  0.3718],
        [ 0.3040, -0.6657, -0.0859],
        [ 0.1781, -0.0364, -0.3391],
        [ 0.3677,  0.0737,  0.2360],
        [ 0.1414, -0.2879,  0.0490],
        [ 0.5518, -0.1114, -0.1928],
        [ 0.5258, -0.3554,  0.3591],
        [ 0.2597,  0.0854, -0.0096]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [ ]:
import torch.nn.functional as F

output = model(input_ids, attention_mask)
probabilities = F.softmax(output, dim=1)

In [ ]:
EPOCHS = 10

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
def train_epoch(
  model,
  data_loader,
  loss_fn,
  optimizer,
  device,
  scheduler,
  n_examples
):
  model = model.train()

  losses = []
  correct_predictions = 0

  for d in data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)

    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )

    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)

    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
%%time

history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)

  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    len(df_train)
  )

  print(f'Train loss {train_loss} accuracy {train_acc}')

  val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn,
    device,
    len(df_val)
  )

  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)

  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc

Epoch 1/10
----------
Train loss 0.7581391797359739 accuracy 0.5733928356048728
Val   loss 0.7296233669223401 accuracy 0.5846348670325031

Epoch 2/10
----------
Train loss 0.7287525869403501 accuracy 0.6105415510794837
Val   loss 0.7266675761882091 accuracy 0.5977205571971296

Epoch 3/10
----------
Train loss 0.6650412310065562 accuracy 0.6817030515016282
Val   loss 0.7798971807796683 accuracy 0.6175601519628535

Epoch 4/10
----------
Train loss 0.5027221543194011 accuracy 0.8019539259437944
Val   loss 0.9540898797896085 accuracy 0.6272688898269312

Epoch 5/10
----------
Train loss 0.37055929825930123 accuracy 0.8737184899288385
Val   loss 1.211334303021431 accuracy 0.6403545799915576

Epoch 6/10
----------
Train loss 0.3013795114755774 accuracy 0.9047159570618744
Val   loss 1.2520535948292522 accuracy 0.642043056141832

Epoch 7/10
----------
Train loss 0.24851520723342063 accuracy 0.9283560487275359
Val   loss 1.6001803776961845 accuracy 0.644153651329675

Epoch 8/10
----------
Train 

In [ ]:
test_acc, _ = eval_model(
  model,
  test_data_loader,
  loss_fn,
  device,
  len(df_test)
)

test_acc.item()

0.6489451476793249

In [ ]:
def get_predictions(model, data_loader):
  model = model.eval()

  review_texts = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in data_loader:

      texts = d["text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      probs = F.softmax(outputs, dim=1)

      review_texts.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(probs)
      real_values.extend(targets)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return review_texts, predictions, prediction_probs, real_values

In [ ]:
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(
  model,
  test_data_loader
)

In [ ]:
print(classification_report(y_test, y_pred, target_names=class_names))

              precision    recall  f1-score   support

    negative       0.61      0.50      0.55       492
     neutral       1.00      0.23      0.37        22
    positive       0.67      0.77      0.72       671

    accuracy                           0.65      1185
   macro avg       0.76      0.50      0.54      1185
weighted avg       0.65      0.65      0.64      1185

